In [1]:
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from os.path import isdir
import numpy as np

In [2]:
def extract_images(filename, required_size=(200, 200)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    image = image.resize(required_size)
    pixels = asarray(image, dtype=np.float32)
    return pixels

In [3]:
def load_images(directory):
    images = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        img = extract_images(path)
        # store
        images.append(img)
    return images


In [4]:
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in ['Covid', 'NORMAL']:
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_images(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [5]:
trainX, trainy = load_dataset('xray_dataset_covid19/')

>loaded 94 examples for class: Covid
>loaded 100 examples for class: NORMAL


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainy = le.fit_transform(trainy)

In [7]:
trainX.shape

(194, 200, 200, 3)

In [8]:
from sklearn.model_selection import train_test_split
trainX, testX, trainy, testy = train_test_split(trainX, trainy, test_size=0.5)

In [9]:
trainX.shape,trainy.shape,testX.shape,testy.shape

((97, 200, 200, 3), (97,), (97, 200, 200, 3), (97,))

# Convolutional Network

Now that we've gotten our feet we with a simple DNN, let's try something more advanced. Although the process is the same, we'll be working with some additional features:
- Convolution, pooling, and dropout layers for building more complex models
- Visualizing training with TensorBoard
- Validation and test set evaluation for measuring generalizability
- Exporting with SavedModel to save training progress and deploy trained models

In [12]:
import time
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}


import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers, metrics

In [12]:
CNN_model = tf.keras.Sequential([
    layers.Reshape(
        target_shape=[200, 200, 3],
        input_shape=(200, 200,3)),
    layers.Conv2D(2, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(4, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dropout(rate=0.4),
    layers.Dense(3)])

optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.5)

In [13]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [14]:
def train_step(model, optimizer, images, labels):

    # Record the operations used to compute the loss, so that the gradient
    # of the loss with respect to the variables can be computed.
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss = compute_loss(labels, logits)
        compute_accuracy(labels, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [15]:
def train(model, optimizer, dataset, log_freq=50):
    """
    Trains model on `dataset` using `optimizer`.
    """
    # Metrics are stateful. They accumulate values and return a cumulative
    # result when you call .result(). Clear accumulated values with .reset_states()
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    # Datasets can be iterated over like any other Python iterable.
    for images, labels in dataset:
        loss = train_step(model, optimizer, images, labels)
        avg_loss(loss)

        if tf.equal(optimizer.iterations % log_freq, 0):
            # summary_ops_v2.scalar('loss', avg_loss.result(), step=optimizer.iterations)
            # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=optimizer.iterations)
            print('step:', int(optimizer.iterations),
                  'loss:', avg_loss.result().numpy(),
                  'acc:', compute_accuracy.result().numpy())
            avg_loss.reset_states()
            compute_accuracy.reset_states()

In [16]:
def test(model, dataset, step_num):
    """
    Perform an evaluation of `model` on the examples from `dataset`.
    """
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    for (images, labels) in dataset:
        logits = model(images, training=False)
        avg_loss(compute_loss(labels, logits))
        compute_accuracy(labels, logits)

    print('Model test set loss: {:0.4f} accuracy: {:0.2f}%'.format(
        avg_loss.result(), compute_accuracy.result() * 100))

    print('loss:', avg_loss.result(), 'acc:', compute_accuracy.result())
    # summary_ops_v2.scalar('loss', avg_loss.result(), step=step_num)
    # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=step_num)

In [17]:
NUM_TRAIN_EPOCHS = 100

for i in range(NUM_TRAIN_EPOCHS):
    start = time.time()
    train_ds = tf.data.Dataset.from_tensor_slices((trainX, trainy))
    train_ds = train_ds.shuffle(200).batch(100)
    #   with train_summary_writer.as_default():
    train(CNN_model, optimizer, train_ds, log_freq=500)
    end = time.time()
    print('Train time for epoch #{} ({} total steps): {}'.format(
        i + 1, int(optimizer.iterations), end - start))
    test_ds = tf.data.Dataset.from_tensor_slices((testX, testy))
    test_ds = test_ds.batch(80)
    test(CNN_model, test_ds, optimizer.iterations)

Train time for epoch #1 (1 total steps): 2.43776798248291
Model test set loss: 1.8741 accuracy: 51.55%
loss: tf.Tensor(1.87413, shape=(), dtype=float32) acc: tf.Tensor(0.5154639, shape=(), dtype=float32)
Train time for epoch #2 (2 total steps): 2.2177484035491943
Model test set loss: 1.5606 accuracy: 52.84%
loss: tf.Tensor(1.5606363, shape=(), dtype=float32) acc: tf.Tensor(0.52835053, shape=(), dtype=float32)
Train time for epoch #3 (3 total steps): 2.021822929382324
Model test set loss: 1.1231 accuracy: 56.01%
loss: tf.Tensor(1.1231394, shape=(), dtype=float32) acc: tf.Tensor(0.56013745, shape=(), dtype=float32)
Train time for epoch #4 (4 total steps): 2.000344753265381
Model test set loss: 1.0319 accuracy: 58.12%
loss: tf.Tensor(1.0318534, shape=(), dtype=float32) acc: tf.Tensor(0.5811856, shape=(), dtype=float32)
Train time for epoch #5 (5 total steps): 2.033550977706909
Model test set loss: 1.0130 accuracy: 61.24%
loss: tf.Tensor(1.013025, shape=(), dtype=float32) acc: tf.Tensor(0.

Model test set loss: 1.1649 accuracy: 76.48%
loss: tf.Tensor(1.1648993, shape=(), dtype=float32) acc: tf.Tensor(0.76481956, shape=(), dtype=float32)
Train time for epoch #41 (41 total steps): 2.6416828632354736
Model test set loss: 1.1763 accuracy: 76.78%
loss: tf.Tensor(1.176276, shape=(), dtype=float32) acc: tf.Tensor(0.7677898, shape=(), dtype=float32)
Train time for epoch #42 (42 total steps): 2.386920928955078
Model test set loss: 1.1638 accuracy: 77.05%
loss: tf.Tensor(1.1638184, shape=(), dtype=float32) acc: tf.Tensor(0.77049583, shape=(), dtype=float32)
Train time for epoch #43 (43 total steps): 2.3940248489379883
Model test set loss: 1.1488 accuracy: 77.28%
loss: tf.Tensor(1.1487575, shape=(), dtype=float32) acc: tf.Tensor(0.77283627, shape=(), dtype=float32)
Train time for epoch #44 (44 total steps): 2.450796127319336
Model test set loss: 1.1717 accuracy: 77.48%
loss: tf.Tensor(1.1716943, shape=(), dtype=float32) acc: tf.Tensor(0.774836, shape=(), dtype=float32)
Train time fo

Train time for epoch #80 (80 total steps): 3.0494930744171143
Model test set loss: 1.4541 accuracy: 81.88%
loss: tf.Tensor(1.4540889, shape=(), dtype=float32) acc: tf.Tensor(0.81875, shape=(), dtype=float32)
Train time for epoch #81 (81 total steps): 2.866715908050537
Model test set loss: 1.4599 accuracy: 81.95%
loss: tf.Tensor(1.4598786, shape=(), dtype=float32) acc: tf.Tensor(0.819524, shape=(), dtype=float32)
Train time for epoch #82 (82 total steps): 2.8500380516052246
Model test set loss: 1.4461 accuracy: 82.02%
loss: tf.Tensor(1.4460534, shape=(), dtype=float32) acc: tf.Tensor(0.82015336, shape=(), dtype=float32)
Train time for epoch #83 (83 total steps): 3.4102540016174316
Model test set loss: 1.4381 accuracy: 82.08%
loss: tf.Tensor(1.4380969, shape=(), dtype=float32) acc: tf.Tensor(0.8207676, shape=(), dtype=float32)
Train time for epoch #84 (84 total steps): 2.620802164077759
Model test set loss: 1.4330 accuracy: 82.15%
loss: tf.Tensor(1.432991, shape=(), dtype=float32) acc: t

In [19]:
CNN_model.save('CNN_model_covid_pneumonia_normal.h5')

# ResNet 

Here we train a tf.keras implementation of ResNet-18

Includes cell dividers for running with IPython!

![](resnet.jpeg)

In [20]:
def conv3x3(channels, stride=1, kernel=(3, 3)):
    return keras.layers.Conv2D(channels, kernel, strides=stride, padding='same',
                               use_bias=False,
                            kernel_initializer=tf.random_normal_initializer())

In [21]:
class ResnetBlock(keras.Model):

    def __init__(self, channels, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()

        self.channels = channels
        self.strides = strides
        self.residual_path = residual_path

        self.conv1 = conv3x3(channels, strides)
        self.bn1 = keras.layers.BatchNormalization()
        self.conv2 = conv3x3(channels)
        self.bn2 = keras.layers.BatchNormalization()

        if residual_path:
            self.down_conv = conv3x3(channels, strides, kernel=(1, 1))
            self.down_bn = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=None):
        residual = inputs

        x = self.bn1(inputs, training=training)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)

        # this module can be added into self.
        # however, module in for can not be added.
        if self.residual_path:
            residual = self.down_bn(inputs, training=training)
            residual = tf.nn.relu(residual)
            residual = self.down_conv(residual)

        x = x + residual
        return x

In [22]:
class ResNet(keras.Model):

    def __init__(self, block_list, num_classes, initial_filters=16, **kwargs):
        super(ResNet, self).__init__(**kwargs)

        self.num_blocks = len(block_list)
        self.block_list = block_list

        self.in_channels = initial_filters
        self.out_channels = initial_filters
        self.conv_initial = conv3x3(self.out_channels)

        self.blocks = keras.models.Sequential(name='dynamic-blocks')

        # build all the blocks
        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):

                if block_id != 0 and layer_id == 0:
                    block = ResnetBlock(self.out_channels, strides=2, residual_path=True)
                else:
                    if self.in_channels != self.out_channels:
                        residual_path = True
                    else:
                        residual_path = False
                    block = ResnetBlock(self.out_channels, residual_path=residual_path)

                self.in_channels = self.out_channels

                self.blocks.add(block)

            self.out_channels *= 2

        self.final_bn = keras.layers.BatchNormalization()
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)

    def call(self, inputs, training=None):

        out = self.conv_initial(inputs)

        out = self.blocks(out, training=training)

        out = self.final_bn(out, training=training)
        out = tf.nn.relu(out)

        out = self.avg_pool(out)
        out = self.fc(out)


        return out

In [24]:
num_classes = 3
batch_size = 32
epochs = 50

# build model and optimizer
ResNet_model = ResNet([2, 2, 2], num_classes)
ResNet_model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
ResNet_model.build(input_shape=(None, 200,200,3))
print("Number of variables in the model :", len(ResNet_model.variables))
ResNet_model.summary()

y_ohe = tf.one_hot(trainy, depth=3).numpy()
testy_ohe = tf.one_hot(testy, depth=3).numpy()

# train
ResNet_model.fit(trainX, y_ohe, batch_size=batch_size, epochs=epochs,
          validation_data=(testX, testy_ohe), verbose=1)

# evaluate on test set
scores = ResNet_model.evaluate(testX, testy_ohe, batch_size, verbose=1)
print("Final test loss and accuracy :", scores)

Number of variables in the model : 77
Model: "res_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           multiple                  432       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  174848    
_________________________________________________________________
batch_normalization_29 (Batc multiple                  256       
_________________________________________________________________
global_average_pooling2d_1 ( multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  195       
Total params: 175,731
Trainable params: 174,707
Non-trainable params: 1,024
_________________________________________________________________
Train on 97 samples, validate on 97 samples
Epoch 1/50
97/97 [=============

Epoch 50/50
97/97 [==============================] - 52s 538ms/sample - loss: 0.1131 - accuracy: 0.9897 - val_loss: 0.3000 - val_accuracy: 0.8969
97/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

# Inception Network

Here we train a tf.keras implementation of InceptionV3


![](inception.png)

In [118]:
class ConvBNRelu(keras.Model):
    
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        
        self.model = keras.models.Sequential([
            keras.layers.Conv2D(ch, kernelsz, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ])
        
        
    def call(self, x, training=None):
        
        x = self.model(x, training=training)
        
        return x 

In [119]:
class InceptionBlk(keras.Model):
    
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        
        self.ch = ch
        self.strides = strides
        
        self.conv1 = ConvBNRelu(ch, strides=strides)
        self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        
        self.pool = keras.layers.MaxPooling2D(3, strides=1, padding='same')
        self.pool_conv = ConvBNRelu(ch, strides=strides)
        
        
    def call(self, x, training=None):
        
        
        x1 = self.conv1(x, training=training)

        x2 = self.conv2(x, training=training)
                
        x3_1 = self.conv3_1(x, training=training)
        x3_2 = self.conv3_2(x3_1, training=training)
                
        x4 = self.pool(x)
        x4 = self.pool_conv(x4, training=training)
        
        # concat along axis=channel
        x = tf.concat([x1, x2, x3_2, x4], axis=3)
        
        return x

In [120]:
class Inception(keras.Model):
    
    def __init__(self, num_layers, num_classes, init_ch=16, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_layers = num_layers
        self.init_ch = init_ch
        
        self.conv1 = ConvBNRelu(init_ch)
        
        self.blocks = keras.models.Sequential(name='dynamic-blocks')
        
        for block_id in range(num_layers):
            
            for layer_id in range(2):
                
                if layer_id == 0:
                    
                    block = InceptionBlk(self.out_channels, strides=2)
                    
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                    
                self.blocks.add(block)
            
            # enlarger out_channels per block    
            self.out_channels *= 2
            
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)
        
        
    def call(self, x, training=None):
        
        out = self.conv1(x, training=training)
        
        out = self.blocks(out, training=training)
        
        out = self.avg_pool(out)
        out = self.fc(out)
        
        return out  

In [123]:
# build model and optimizer
batch_size = 32
epochs = 100
Inception_model = Inception(2, 3)
# derive input shape for every layers.
Inception_model.build(input_shape=(None, 200, 200, 3))
Inception_model.summary()

optimizer = keras.optimizers.Adam(learning_rate=1e-3)
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)

acc_meter = keras.metrics.Accuracy()

db_train = tf.data.Dataset.from_tensor_slices((trainX, trainy)).batch(256)
db_test = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(256)


for epoch in range(100):

    for step, (x, y) in enumerate(db_train):

        with tf.GradientTape() as tape:
            # print(x.shape, y.shape)
            # [b, 10]
            logits = Inception_model(x)
            # [b] vs [b, 10]
            loss = criteon(tf.one_hot(y, depth=3), logits)

        grads = tape.gradient(loss, Inception_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, Inception_model.trainable_variables))

        if step % 10 == 0:
            print(epoch, step, 'loss:', loss.numpy())


    acc_meter.reset_states()
    for x, y in db_test:
        # [b, 10]
        logits = Inception_model(x, training=False)
        # [b, 10] => [b]
        pred = tf.argmax(logits, axis=1)
        # [b] vs [b, 10]
        acc_meter.update_state(y, pred)

    print(epoch, 'evaluation acc:', acc_meter.result().numpy())

Model: "inception_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu_42 (ConvBNRelu) multiple                  512       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  292704    
_________________________________________________________________
global_average_pooling2d_9 ( multiple                  0         
_________________________________________________________________
dense_19 (Dense)             multiple                  387       
Total params: 293,603
Trainable params: 292,611
Non-trainable params: 992
_________________________________________________________________
0 0 loss: 4.1779428
0 evaluation acc: 0.3265306
1 0 loss: 43.183533
1 evaluation acc: 0.34013605
2 0 loss: 20.469946
2 evaluation acc: 0.3265306


KeyboardInterrupt: 

# VGG16

Fitting a VGG-16 network for image classification!
We use gradient clipping for faster convergence.

A complete implementation of VGG-16 is available in network.py

![](vgg16.png)

In [10]:
from tensorflow.keras import regularizers

In [13]:
class VGG16(models.Model):


    def __init__(self, input_shape, num_classes):
        """

        :param input_shape: [32, 32, 3]
        """
        super(VGG16, self).__init__()

        weight_decay = 0.000
        self.num_classes = num_classes

        model = models.Sequential()

        model.add(layers.Conv2D(64, (3, 3), padding='same',
                         input_shape=input_shape, kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(self.num_classes))
        # model.add(layers.Activation('softmax'))


        self.model = model


    def call(self, x):

        x = self.model(x)

        return x

In [14]:
def compute_loss(logits, labels):
    return tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits, labels=labels))

In [17]:
VGG16_model = VGG16([200, 200, 3], 3)


# must specify from_logits=True!
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)
metric = keras.metrics.CategoricalAccuracy()

optimizer = optimizers.Adam(learning_rate=0.0001)

db_train = tf.data.Dataset.from_tensor_slices((trainX, trainy)).batch(256)
db_test = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(256)

for epoch in range(200):

    for step, (x, y) in enumerate(db_train):
        # [b, 1] => [b]
        # y = tf.squeeze(y, axis=1)
        # [b, 10]
        y = tf.one_hot(y, depth=3)

        with tf.GradientTape() as tape:
            logits = VGG16_model(x)
            loss = criteon(y, logits)
            # loss2 = compute_loss(logits, tf.argmax(y, axis=1))
            # mse_loss = tf.reduce_sum(tf.square(y-logits))
            # print(y.shape, logits.shape)
            metric.update_state(y, logits)

        grads = tape.gradient(loss, VGG16_model.trainable_variables)
        # MUST clip gradient here or it will disconverge!
        grads = [ tf.clip_by_norm(g, 15) for g in grads]
        optimizer.apply_gradients(zip(grads, VGG16_model.trainable_variables))

        if step % 40 == 0:
            # for g in grads:
            #     print(tf.norm(g).numpy())
            print(epoch, step, 'loss:', float(loss), 'acc:', metric.result().numpy())
            metric.reset_states()


    if epoch % 1 == 0:

        metric = keras.metrics.CategoricalAccuracy()
        for x, y in db_test:
            # [b, 1] => [b]
            # y = tf.squeeze(y, axis=1)
            # [b, 10]
            y = tf.one_hot(y, depth=3)

            logits = VGG16_model.predict(x)
            # be careful, these functions can accept y as [b] without warnning.
            metric.update_state(y, logits)
        print('test acc:', metric.result().numpy())
        metric.reset_states()

0 0 loss: 0.9745582342147827 acc: 0.5154639
test acc: 0.48453608
1 0 loss: 1.7481777667999268 acc: 0.48453608
test acc: 0.5154639
2 0 loss: 3.1571853160858154 acc: 0.5154639
test acc: 0.5154639
3 0 loss: 0.7141163349151611 acc: 0.5154639
test acc: 0.48453608
4 0 loss: 0.8749623894691467 acc: 0.48453608
test acc: 0.48453608
5 0 loss: 0.6982256174087524 acc: 0.48453608
test acc: 0.9587629
6 0 loss: 0.6503109335899353 acc: 0.91752577
test acc: 0.5154639
7 0 loss: 0.638228178024292 acc: 0.5154639
test acc: 0.5463917
8 0 loss: 0.6018865704536438 acc: 0.57731956
test acc: 0.9484536
9 0 loss: 0.5470068454742432 acc: 0.92783505
test acc: 0.8969072
10 0 loss: 0.4932529330253601 acc: 0.8556701
test acc: 0.96907216
11 0 loss: 0.42065703868865967 acc: 0.92783505
test acc: 0.9072165
12 0 loss: 0.33522695302963257 acc: 0.88659793
test acc: 0.96907216
13 0 loss: 0.29068467020988464 acc: 0.92783505
test acc: 0.6494845
14 0 loss: 0.5618012547492981 acc: 0.77319586
test acc: 0.92783505
15 0 loss: 0.3002

KeyboardInterrupt: 

In [18]:
tf.saved_model.save(VGG16_model,'VGG16_Model_Covid_Normal')

INFO:tensorflow:Assets written to: VGG16_Model_Covid_Normal/assets
